In [1]:
import pandas as pd
from utils import secrets
import mysql.connector
from tldextract import extract

In [2]:
db = mysql.connector.connect(
        host = 'localhost',
        user = 'root',
        password = secrets.MYSQL_PWD,
        database = 'MapTheInternet',
        autocommit = True
    )

In [3]:
domains = pd.read_sql(sql='SELECT * FROM domains', con=db)

/var/folders/zl/1tvb16z55nl5z3qjjlv168lr0000gn/T/ipykernel_16629/2300359958.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  domains = pd.read_sql(sql='SELECT * FROM domains', con=db)


In [4]:
domains.drop('id', axis=1, inplace=True)

In [5]:
# replace the http to group by domains in case different websites used different protocols to connect to the same website (http, https)
domains['domain'] = domains['domain'].str.replace('https://', '')
domains['domain'] = domains['domain'].str.replace('http://', '')

domains = domains.groupby(['domain'], as_index=False).sum()

/var/folders/zl/1tvb16z55nl5z3qjjlv168lr0000gn/T/ipykernel_16629/3262769879.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  domains = domains.groupby(['domain'], as_index=False).sum()


In [6]:
# clean up damaged links
blankIndices = domains[domains['domain'].str.contains(' ')].index
domains.drop(blankIndices, inplace=True)

damagedIndices = domains[domains['domain'].str.startswith('.')].index
domains.drop(damagedIndices, inplace=True)

In [7]:
# extract suffixes (top level domain) for filtering and visualization
domains['suffix'] = domains['domain'].str.split('.').str[-1]
domains = domains.mask(domains == '')
domains.dropna(inplace=True)

In [8]:
domains = domains.sort_values(by='numIncomingLinks', ascending=False)

In [9]:
domains

,domain,numIncomingLinks,numInternalLinks,suffix
930739,twitter.com,425817,6,com
308108,facebook.com,376837,18,com
454525,instagram.com,261462,0,com
1029031,youtube.com,220743,7,com
45009,adesignaward.com,216477,0,com
...,...,...,...,...
712116,pokpokrestaurants.com,1,0,com
712115,pokourenicko.cz,1,0,cz
338595,freemalecelebrityarchive.com,1,0,com
4,\r\ntzshuo.com,1,0,com


### Explanation of the columns:
- domain: the url of a website
  - https://www.google.com/search? has google.com and so does https://drive.google.com/
- numIncomingLins: the number of times other websites linked to this website
  - if https://www.facebook.com/marketplace/?ref=bookmark has a link to  https://drive.google.com/ then the numIncomingLinks of google.com is increased by 1
- numInternalLinks: the number of time a page has a link to another page with the same domain
  - if https://drive.google.com/ has a link to https://www.google.com/search? then the numIntenalLinks of google.com is increased by 1 since bothe pages are from the same domain
- suffix: the top level domain of the website
  - google.com has 'com' as a suffix

In [10]:
domains.to_csv('data/domains.csv', index=False)